# 第6章-最大熵模型

参考
* [A Brief Maxent Tutorial](http://www.cs.cmu.edu/afs/cs/user/aberger/www/html/tutorial/tutorial.html)
* 李航: 《统计学习方法》

## 1. Overview

为了对随机过程建模，我们往往需要利用该随机过程产生的一些数据。这些数据，虽然不完备，但是仍然包含了关于整个随机过程的一些信息(an incomplete state of knowledge about the process)。建模，就是将这些信息有效地提取出来，形成一个简洁、准确的表达，使得我们能够利用其对未来进行预测。

指数族模型(Exponential models)很实用，最大熵模型(Maximum entropy model)属于其中。

该模型在NLP十分常用。

### 1) Motivating example

什么是最大熵？

试着想象，我们希望模拟一位翻译专家，能够决定将英国中的“in”应该翻译成哪一个对应的法语词汇。

我们用$p$来代表这个模型，$p$能够为每一个法语词$f$汇分配一个估计值$p(f)$，表示应该选它作为“in”的翻译的概率。

为了训练$p$，我们必须从翻译专家那里收集一些数据，用这些数据：
1. extract a set of facts about the decision-making process: 抽取一些关于决策过程的事实；
2. aid us in constructing a model of this process: 基于这些事实构建模型。

例如，我们从训练数据中发现，“in”在法语中通常会被翻译成5个意思，{dans, en, à, au cours de, pendant}。通过这个信息，我们可以为模型$p$加上第一个约束:

$$p(dans)+p(en)+p(à)+p(au cours de)+p(pendant)=1$$

现在，我们可以寻找满足以上约束的model。但注意，这样的model有无数个，比如：

* $p(dans)=1$，表示一直把“in“翻译成”dans“；
* $p(en)=\frac{1}{2}，p(à)=\frac{1}{2}$；

但是上面的两个模型都有些违背了我们的直觉。因为我们可能会问，在没有任何其他知识的基础上，凭什么多选"dans"或"en"……因此，在有限的知识下，”平均“的模型是最好的，即：

* $p(dans)=1/5$
* $p(en)=1/5$
* $p(à)=1/5$
* $p(au cours de)=1/5$
* $p(pendant)=1/5$

接着，我们从训练数据中发现了更多的线索。例如，我们注意到，训练数据中，"dans"和"en"被选中的频率之和为30%(the expert chose either dans or en 30% of the time)。我们可能认为，这个事实很重要，希望我们的模型$p$也满足这样的条件。因此，我们可以再添加一个约束，从而得到两个约束：

$$p(dans)+p(en)+p(à)+p(au cours de)+p(pendant)=1$$

$$p(dans)+p(en)=3/10$$

同样，满足上述两个约束的模型还是有无限个。在没有其他任何知识的情况下，永远记住，最均匀的模型是最好的。因此，我们会选择如下的$p$：

* $p(dans)=3/20$
* $p(en)=3/20$
* $p(à)=7/30$
* $p(au cours de)=7/30$
* $p(pendant)=7/30$

再一次检视数据，这次又发现，一半的数据里，专家将“in”翻译成“dans”或“à”。因此，我们有理由相信，我们的模型也满足这样的特点。因此，我们可以为模型$p$添加第三个约束，此时：

$$p(dans)+p(en)+p(à)+p(au cours de)+p(pendant)=1$$

$$p(dans)+p(en)=3/10$$

$$p(dans)+p(à)=1/2$$

此时，我们还是要选择满足上述所有约束的模型中，“最平均”的那一个。但此时，这个“最平均”的那个不是那么明显了。我们需要知道：

1. 如何描述一个model的平均程度(uniformity of a model)?
2. 定义了uniformity后，如何在约束条件下，选到最平均的模型?

最大熵方法(The maximum entropy method)回答了上述两个问题。基于的思想是——model all that is known and assume nothing about that which is unknown，也就是given a collection of facts, choose a model which is consistent with all the facts, but otherwise as uniform as possible.

## 2. Maxent Modeling

考虑一个随机过程，每次输出$y \in \mathcal{Y}$。对于刚才的问题，$\mathcal{Y} = \{dans, en, à, au cours de, pendant\}$。

容易想到，这个random process并不是完全random的，其每次的输出$y$，都会受到一些影响，如上下文信息(some contextual information)，我们记做$x \in \mathcal{X}$。例如，每次的$x$是“in”周围的单词(the words in the English sentence surrounding "in")。

我们的任务是训练这样一个模型$p(Y|X)$，能够输出在给定上下文信息$x$的情况下输出$y$的条件概率(estimating the conditional probability that, given a context x, the process will output y)，即给到$p(Y=y|X=x)$.

一些记号说明：

* $X$是随机变量，$x$是具体的取值；
* $Q(X)$是概率分布，$q(X=x)$是具体的随机变量取某个值时的概率——也可以简写成$q(x)$——an entire model $Q(X)$ or the value assigned by the model to the event $X=x$；
* $P$记做所有条件概率分布的集合，因此模型$p(y|x)$只是$P$的一个元素。

### 1) Training data

Samples: $(x_1,y_1),(x_2,y_2),\cdots,(x_N,y_N)$

* $x$: a phrase x containing the words surrounding "in";
* $y$: the translation y of in which the process produced.

对于上面的training data，我们可以计算出经验概率分布$\tilde{p}$：

$$\tilde{p}(x,y) = \frac{1}{N} × number\ of\ times\ that\ (x,y)\ occurs\ in\ the\ sample$$

注意，对于一个特定的$(x,y)$，可能没有出现在sample中，也可能出现了好几次。

### 2) Features and constraints

我们当下目标是：为随机过程(该过程生成了经验分布为$\tilde{p}(x,y)$的训练数据)构建一个模型。我们需要从训练数据中抽取一系列的统计知识来构建模型。我们之前抽取的知识有：

* “in”被翻译成“dans”或“en”的频率为3/10；
* “in”被翻译成“dans”或“au cours de”的频率为1/2；
* ……

但注意到，上述的知识，都是独立于上下文的。我们当然可以提取一些依赖于上下文的知识(statistics which depend on the conditioning information $x$)。例如，我们注意到，在训练数据中，如果"April"是"in"的下一个单词，那么“in”被翻译成“en”的频率为9/10。

如果要描述这样的有依赖关系的知识，我们需要引入指示函数(indicator function)：

\begin{equation}
f(x,y)=
\begin{cases}
1& \text{if y=en and April follows in}\\
0& \text{otherwise}
\end{cases}
\end{equation}

这里，$f$关于经验概率分布$\tilde{p}(x,y)$的期望是我们感兴趣的统计指标，我们记做：

$$\tilde{p}(f) = \sum_{x,y} \tilde{p}(x,y)f(x,y)$$

我们将上面的指示函数，称为特征函数(feature function)，或者直接叫做特征。

每当我们觉得一个“特征”比较重要，就可以“强制”要求我们的模型必须“符合它”(accord with it)。

首先，我们可以列出我们的模型$p$中特征$f$的期望(the expected value that the model assigns to the corresponding feature function f)：

\begin{align*}
 p(f) &= \sum_{x,y}p(x,y)·f(x,y)\\
      &= \sum_{x,y}p(x)·p(y|x)·f(x,y)\\
\end{align*}

由于我们不知道$p(x)$，因此我们用经验分布$\tilde{p}(x)$代替，有：

$$p(f) = \sum_{x,y}\tilde{p}(x)·p(y|x)·f(x,y)$$

这时，我们通过下式，“强制”要求我们的模型必须“符合它”：

$$p(f) = \tilde{p}(f)$$

即：

$$\sum_{x,y}\tilde{p}(x)·p(y|x)·f(x,y) = \sum_{x,y} \tilde{p}(x,y)·f(x,y)$$

我们将上式称为一个约束方程(constraint equation)，简称约束(constraint)。这样，我们就从一大堆模型中，根据特征提供的知识，排除掉了一些不符合约束的模型。

### 3) The maxent principle

假如我们一共提取了$n$个特征$f_i$，我们希望我们的model能够符合它们。也可以说：

$$C = \Big\{ p\in P\ |\ p(f_i)=\tilde{p}(f_i)\ for\ i \in \{1,2,\cdots,n\} \Big\}$$

我们将从$C$中选出来我们想要的模型(注意，$C$一般包含了无限个模型)。

最大熵原理告诉我们，要在$C$中挑选最平均(uniform)的模型。怎么样度量一个模型的平均程度呢？一个度量某条件分布$p(Y|X)$平均程度的数学方法是计算其__条件熵(conditional entropy)__：

$$H(p) = -\sum_{x,y}\tilde{p}(x)·p(y|x)·log\ p(y|x)$$

此时，我们将选择一个模型，满足$p^*\in C$，同时有着最大的条件熵$H(p)$：

$$p^*=\underset{p \in C}{argmax}\ H(p)$$

这样的模型$p^*$是唯一的。

### 4) Exponential form

根据最大熵原理，我们将模型构建的问题具体为一个在一些约束条件下的最优化问题：

* 等式约束：关于我们认为的重要的特征，在样本分布上的期望=我们模型分布上的期望；
* 最大化熵：我们将平均程度转化为熵的概念，熵越大，越平均。

因此，我们需要运用拉格朗日乘子法来一般化地求解以下问题(将最大化改为最小化，加上一个负号)：

\begin{align*}
              & \underset{p \in \mathbf{C}}{min}\ H(p) =\sum_{x,y}\tilde{p}(x)·p(y|x)·log\ p(y|x)\\
    s.t.\ \ \ & p(y|x) \ge 0,\quad for\ all\ x,y\\
              & \sum_yp(y|x) = 1,\quad for\ all\ x\\
              & \sum_{x,y}\tilde{p}(x)·p(y|x)·f(x,y) = \sum_{x,y} \tilde{p}(x,y)·f(x,y),\quad i=1,2,\cdots,n
\end{align*}

首先，我们引入拉格朗日乘子，构造拉格朗日函数：

\begin{align*}
 \xi(p,\Lambda, \gamma) &=\sum_{x,y}\tilde{p}(x)·p(y|x)·log\ p(y|x) \\
                        &+ \sum_i \lambda_i\Big(\sum_{x,y} \tilde{p}(x,y)·f_i(x,y) -\tilde{p}(x)·p(y|x)·f_i(x,y)\Big)\\
                        &+ \gamma\Big(\sum_yp(y|x)-1\Big)
\end{align*}

上式中，$\gamma$与$\Lambda=\{\lambda_1,\lambda_2,\cdots,\lambda_n\}$是拉格朗日乘子，对应共n+1个约束条件。

原始问题(primal)是：

$$\underset{p}{min}\ \underset{\Lambda,\gamma}{max}\ \xi(p,\Lambda, \gamma)$$

对偶问题(dual)是：

$$\underset{\Lambda,\gamma}{max}\ \underset{p}{min}\ \xi(p,\Lambda, \gamma)$$

可以证明，这里primal和dual是等价的。因此，我们选择解dual。

__1. 对p求偏导__

$$\frac{\partial \xi}{\partial p(y|x)} =\sum_{x,y}\tilde{p}(x)\Big(1+log\ p(y|x)-\sum_i\lambda_if_i(x,y)+\gamma\Big)$$

__2. 令偏导等于0，初步解出$p$(用$\Lambda$和$\gamma$表示的)__

因为$\tilde{p}(x) \ge 0$，所以零导函数等于0，得到：

$$1+log\ p(y|x)-\sum_i\lambda_if_i(x,y)+\gamma=0$$

得到：

$$p_{\Lambda,\gamma}(y|x) = exp\Big(\sum_i\lambda_if_i(x,y)\Big)exp\Big(-\gamma-1\Big)$$

__3. 运用概率和为1的约束，进一步将$p^*$转化__

由于$\sum_y p(y|x)=1$，因此有$\sum_y p_{\Lambda,\gamma}(y|x)=1$，即：

$$\sum_y exp\Big(\sum_i\lambda_if_i(x,y)\Big)exp\Big(-\gamma-1\Big)=1$$

故：

$$exp(-\gamma-1) = \frac{1}{\sum_yexp\Big(\sum_i \lambda_if_i(x,y)\Big)}$$

将上式代入$p_{\Lambda,\gamma}(y|x)$中，得到：

$$p_\Lambda(y|x)=\frac{1}{Z_\Lambda(x)}exp\Big(\sum_i\lambda_if_i(x,y)\Big)$$

其中，记规范化因子(normalizing factor)：

$$Z_\Lambda(x)=\sum_yexp\Big(\sum_i \lambda_if_i(x,y)\Big)$$

__上式便是最大熵模型中，目标条件概率$p$的形式，$\Lambda$是参数。__

此时，我们将对偶问题的内层解了出来，下面需要解外层的最大化问题：

$$\underset{\Lambda}{max}\ \xi(p^*,\Lambda, \gamma^*)$$

引入记号(对偶函数，dual function)：

$$\Psi(\Lambda)=\xi(p^*,\Lambda,\gamma^*)$$

现在我们的问题是：

$$\Lambda^* = \underset{\Lambda}{argmax}\Psi(\Lambda)$$

### 5) Maximum likelihood

下面我们暂时从最大似然估计的角度看该问题。

在已知训练数据的经验分布$\tilde{p}(X,Y)$时，条件概率分布$p(Y|X)$的对数似然函数为：

$$L_{\tilde{p}}(p)=log \prod_{x,y}p(y|x)^{\tilde{p}(x,y)} = \sum_{x,y}\tilde{p}(x,y)log\ p(y|x)$$

将最大熵模型中$p$的表示$p_{\Lambda}(y|x)$(parametric form)方法代入上式，得到:

$$L_{\tilde{p}}(p)=L_{\tilde{p}}(p_{\Lambda}(y|x))=L_{\tilde{p}}(\Lambda)=\Psi(\Lambda)$$

因此，最大化对偶函数，等价于，最大化对数似然函数：

"The model $p^* \in C$ with maximum entropy is the model in the parametric family $p(y|x)$ that maximizes the likelihood of the training sample $\tilde{p}$".

换句话说，最大熵模型等价于满足一些条件下的最大似然估计：

1. 似然函数是对数函数；
2. 条件概率分布满足$p_\Lambda(y|x)$的形式。

## 3. Computing the Parameters

### 1) 问题再述

我们从最大似然估计的角度出发，写出等价于最大熵模型的优化问题：

$$max\ L_{\tilde{p}}(p)=log \prod_{x,y}p(y|x)^{\tilde{p}(x,y)} = \sum_{x,y}\tilde{p}(x,y)log\ p(y|x)$$

其中：

$$p(y|x)=\frac{1}{Z_\Lambda(x)}exp\Big(\sum_i\lambda_if_i(x,y)\Big)$$

$$Z_\Lambda(x)=\sum_yexp\Big(\sum_i \lambda_if_i(x,y)\Big)$$

我们可以进一步写成：

$$max\ L_{\tilde{p}}(\Lambda) = \sum_{x,y}\tilde{p}(x,y)\sum_i\lambda_if_i(x,y)-\sum_x\tilde{p}(x)log\ Z_\Lambda(x)$$

其中：

$$Z_\Lambda(x)=\sum_yexp\Big(\sum_i \lambda_if_i(x,y)\Big)$$

上面的最优化问题，仅有一个变量$\Lambda$，我们可以使用：

* 梯度下降法；
* 改进的迭代尺度法；
* 拟牛顿法；

解决。

### 2) IIS算法

有一种专门为解决最大熵问题设计的优化算法——__改进的迭代尺度算法(Improved Iterative Scaling)__。该算法应用的条件是，特征函数非负，即：

$$f_i(x,y)\ge0,\quad for\ all\ i,x,y$$

![IIS](./pic/ch6-1.png)